In [1]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
import numpy as np
import string
import re
import os
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Convert .parquet to .txt

In [3]:
df = pd.read_parquet('/content/drive/MyDrive/0000.parquet')

# Define the path where you want to save the .txt file
txt_file_path = '/content/drive/MyDrive/output.txt'

# Write the DataFrame to a .txt file
df.to_csv(txt_file_path, sep='\t', index=False)

print(f"File '{txt_file_path}' created successfully.")

File '/content/drive/MyDrive/output.txt' created successfully.


In [4]:
text_file = "/content/drive/My Drive/output.txt"
with open(text_file) as f:
  lines = f.read().split("\n")[:-1]

i = 0
for line in lines:
  print(line)
  i = i + 1
  if(i==20):
    break

english	sinhala
you will receive a package in the mail	ඔයාට තැපෑලෙන් පැකේජ් එකක් හම්බවේවි
you will receive a confirmation code after completing the registration	ලියාපදිංචිය සම්පූර්ණ කලාට පස්සෙ ඔයාට තහවුරු කිරීමේ කේතයක්  හම්බවේවි
you will receive a discount on your next purchase	ඊලඟ මිලදී ගැනීම කරන කොට ඔයාට වට්ටමක් හම්බවේවි
you will receive a phone call from our customer service team	ඔයාට අපේ පාරිභෝගික සේවා කණ්ඩායමෙන් දුරකථන ඇමතුමක් හම්බවේවි
you will receive a notification when your order is ready for pickup	ඔයාගේ ඇණවුම සූදානම් උනාට පස්සෙ ඔයාට දැනුම් දීමක් හම්බවේවි
you will receive a response to your inquiry within 24 hours	ඔයාට පැය 24ක් ඇතුලත ඔයාගෙ විමසීමට පිළිතුරක්  හම්බවේවි
you will receive a gift for your loyalty	ඔයාට ඔයාගේ පක්ෂපාතිත්වය වෙනුවෙන් තෑග්ගක් හම්බවේවි
you will receive an invitation to the event	ඔයාට උත්සවයට ආරාධනාවක් හම්බවේවි
you will receive a refund for the returned item	ඔයාට රිටන් කරන භාණ්ඩය වෙනුවෙන් ආපසු මුදල් ගෙවීමක් හම්බවේවි
you'll receive an email with the details	

In [5]:
for x in range(len(lines)-10,len(lines)):
  print(lines[x])

Once upon a time in a small village there lived a young girl named Lily who had a special gift for growing the most beautiful flowers	එක් කලෙක කුඩා ගමක ලිලී නම් තරුණියක් ජීවත් වූ අතර ඇයට ලස්සනම මල් වගා කිරීම සඳහා විශේෂ තෑග්ගක් තිබුණි
In a distant land a brave knight named Sir Arthur embarked on a perilous journey to rescue a kidnapped princess from an evil sorcerer	දුර රටක ශ්‍රීමත් ආතර් නම් නිර්භීත නයිට්වරයා නපුරු මායාකාරයෙකුගෙන් පැහැරගෙන ගිය කුමරියක් බේරා ගැනීමට භයානක ගමනක් ආරම්භ කළේය
Emily a curious explorer set out on an adventure to uncover the hidden treasures of an ancient lost city deep in the Amazon rainforest	කුතුහලය දනවන ගවේෂකයෙකු වන එමිලි ඇමසන් වනාන්තරයේ ගැඹුරින් ගිලිහී ගිය පැරණි නගරයක සැඟවුණු වස්තු සොයා ගැනීමට ත්‍රාසජනක ගමනක් ආරම්භ කළාය
In the peaceful town of Willowbrook a mischievous cat named Oliver had a talent for getting into amusing and unexpected predicaments	සාමකාමී නගරයක් වන විලෝබෲක්හි ඔලිවර් නම් දඟකාර බළලාට විනෝදජනක සහ අනපේක්ෂිත දුෂ්කරතාවන්ට පත්වීමේ දක්ෂතාවයක් ති

In [6]:
text_pairs = []
for line in lines:
  english, sinhala = line.split("\t")
  sinhala = "[start] " + sinhala + " [end]"
  text_pairs.append((english, sinhala))

for i in range(3):
  print(random.choice(text_pairs))

('if i ever find out that you lied to me i will be very sad son', '[start] මම කවදහරි දැනගත්තොත් ඔයා මට බොරු කිව්වා කියලා ගොඩක් දුක හිතෙයි පුතේ [end]')
('a long time ago', '[start] ගොඩක් ඉස්සර [end]')
('you take the leg', '[start] ඔයා කකුලුවාව ගන්න [end]')


In [7]:
import random
random.shuffle(text_pairs)

Splitting the data into training,testing and validation

In [8]:
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples:]

print("Total sentences:",len(text_pairs))
print("Training set size:",len(train_pairs))
print("Validation set size:",len(val_pairs))
print("Testing set size:",len(test_pairs))

Total sentences: 80685
Training set size: 56481
Validation set size: 12102
Testing set size: 12102


In [9]:
len(train_pairs)+len(val_pairs)+len(test_pairs)

80685

In [10]:
strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[","")
strip_chars = strip_chars.replace("]","")
f"[{re.escape(strip_chars)}]"



'[!"\\#\\$%\\&\'\\(\\)\\*\\+,\\-\\./:;<=>\\?@\\\\\\^_`\\{\\|\\}\\~¿]'

In [11]:
f"{3+5}"

'8'

In [12]:
def custom_standardization(input_string):
  lowercase = tf.strings.lower(input_string)
  return tf.strings.regex_replace(
    lowercase, f"[{re.escape(strip_chars)}]","")
vocab_size = 15000
sequence_length = 20

source_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)
target_vectorization = layers.TextVectorization(
  max_tokens=vocab_size,
  output_mode="int",
  output_sequence_length=sequence_length + 1,
  standardize=custom_standardization,
)
train_english_texts = [pair[0] for pair in train_pairs]
train_sinhala_texts = [pair[1] for pair in train_pairs]
source_vectorization.adapt(train_english_texts)
target_vectorization.adapt(train_sinhala_texts)

In [13]:
batch_size = 64
def format_dataset(eng, sin):
  eng = source_vectorization(eng)
  sin = target_vectorization(sin)
  return ({
    "english": eng,
    "sinhala": sin[:, :-1],
}, sin[:, 1:])

def make_dataset(pairs):
  eng_texts, sin_texts = zip(*pairs)
  eng_texts = list(eng_texts)
  sin_texts = list(sin_texts)
  dataset = tf.data.Dataset.from_tensor_slices((eng_texts, sin_texts))
  dataset = dataset.batch(batch_size)
  dataset = dataset.map(format_dataset, num_parallel_calls=4)
  return dataset.shuffle(2048).prefetch(16).cache()
train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)
for inputs, targets in train_ds.take(1):
  print(f"inputs['english'].shape: {inputs['english'].shape}")
  print(f"inputs['sinhala'].shape: {inputs['sinhala'].shape}")
  print(f"targets.shape: {targets.shape}")

  inputs['english'].shape: (64, 20)
  inputs['sinhala'].shape: (64, 20)
  targets.shape: (64, 20)

print(list(train_ds.as_numpy_iterator())[50])

inputs['english'].shape: (64, 20)
inputs['sinhala'].shape: (64, 20)
targets.shape: (64, 20)
({'english': array([[  320,    24,   533, ...,     0,     0,     0],
       [ 5273,    19,  4932, ...,     0,     0,     0],
       [   46,    74,    80, ...,     0,     0,     0],
       ...,
       [  340, 11477,  8129, ...,     0,     0,     0],
       [  104,     2,     0, ...,     0,     0,     0],
       [    3,   420,     5, ...,     0,     0,     0]]), 'sinhala': array([[   2,   17,  477, ...,    0,    0,    0],
       [   2, 7915,  172, ...,    0,    0,    0],
       [   2,    7,   66, ...,    0,    0,    0],
       ...,
       [   2,    1,    1, ...,    0,    0,    0],
       [   2,  534,    3, ...,    0,    0,    0],
       [   2,    4,   78, ...,    0,    0,    0]])}, array([[   17,   477,  1792, ...,     0,     0,     0],
       [ 7915,   172, 11525, ...,     0,     0,     0],
       [    7,    66,     3, ...,     0,     0,     0],
       ...,
       [    1,     1,  2776, ...,     0

Transformer encoder implemented as a subclassed Layer

In [14]:
class TransformerEncoder(layers.Layer):
  def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
    super().__init__(**kwargs)
    self.embed_dim = embed_dim
    self.dense_dim = dense_dim
    self.num_heads = num_heads
    self.attention = layers.MultiHeadAttention(
    num_heads=num_heads, key_dim=embed_dim)
    self.dense_proj = keras.Sequential(
    [layers.Dense(dense_dim, activation="relu"),
    layers.Dense(embed_dim),]
    )
    self.layernorm_1 = layers.LayerNormalization()
    self.layernorm_2 = layers.LayerNormalization()

  def call(self, inputs, mask=None):
    if mask is not None:
      mask = mask[:, tf.newaxis, :]
    attention_output = self.attention(
      inputs, inputs, attention_mask=mask)
    proj_input = self.layernorm_1(inputs + attention_output)
    proj_output = self.dense_proj(proj_input)
    return self.layernorm_2(proj_input + proj_output)

  def get_config(self):
    config = super().get_config()
    config.update({
      "embed_dim": self.embed_dim,
      "num_heads": self.num_heads,
      "dense_dim": self.dense_dim,
    })
    return config

The Transformer decoder

In [15]:
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1),
             tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(
                mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
        else:
            padding_mask = mask
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=causal_mask)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        attention_output_2 = self.layernorm_2(
            attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)

Positional Encoding

In [16]:
class PositionalEmbedding(layers.Layer):
  def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
    super().__init__(**kwargs)
    self.token_embeddings = layers.Embedding(
      input_dim=input_dim, output_dim=output_dim)
    self.position_embeddings = layers.Embedding(
      input_dim=sequence_length, output_dim=output_dim)
    self.sequence_length = sequence_length
    self.input_dim = input_dim
    self.output_dim = output_dim

  def call(self, inputs):
    length = tf.shape(inputs)[-1]
    positions = tf.range(start=0, limit=length, delta=1)
    embedded_tokens = self.token_embeddings(inputs)
    embedded_positions = self.position_embeddings(positions)
    return embedded_tokens + embedded_positions

  def compute_mask(self, inputs, mask=None):
    return tf.math.not_equal(inputs, 0)

  def get_config(self):
    config = super(PositionalEmbedding, self).get_config()
    config.update({
    "output_dim": self.output_dim,
    "sequence_length": self.sequence_length,
    "input_dim": self.input_dim,
    })
    return config


In [17]:
embed_dim = 256
dense_dim = 2048
num_heads = 8

# Define encoder inputs
encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="english")

# Add positional embedding to encoder inputs
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)

# Encode input sequence
encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)

# Define decoder inputs
decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="sinhala")

# Add positional embedding to decoder inputs
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)

# Decode input sequence
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs)

# Apply dropout
x = layers.Dropout(0.5)(x)

# Generate decoder outputs
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)

# Define the transformer model
transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)
transformer.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 english (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 sinhala (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 positional_embedding (Posi  (None, None, 256)            3845120   ['english[0][0]']             
 tionalEmbedding)                                                                                 
                                                                                                  
 positional_embedding_1 (Po  (None, None, 256)            3845120   ['sinhala[0][0]']         

In [18]:
transformer.compile(
  optimizer="rmsprop",
  loss="sparse_categorical_crossentropy",
  metrics=["accuracy"])
transformer.fit(train_ds, epochs=10, validation_data=val_ds)

Epoch 1/10
883/883 [==============================] - 83s 83ms/step - loss: 4.6883 - accuracy: 0.4019 - val_loss: 3.8546 - val_accuracy: 0.4683
Epoch 2/10
883/883 [==============================] - 61s 69ms/step - loss: 3.9213 - accuracy: 0.4673 - val_loss: 3.5263 - val_accuracy: 0.5006
Epoch 3/10
883/883 [==============================] - 60s 68ms/step - loss: 3.6154 - accuracy: 0.4955 - val_loss: 3.3727 - val_accuracy: 0.5147
Epoch 4/10
883/883 [==============================] - 60s 68ms/step - loss: 3.4335 - accuracy: 0.5150 - val_loss: 3.3356 - val_accuracy: 0.5185
Epoch 5/10
883/883 [==============================] - 60s 68ms/step - loss: 3.2984 - accuracy: 0.5329 - val_loss: 3.2778 - val_accuracy: 0.5289
Epoch 6/10
883/883 [==============================] - 60s 68ms/step - loss: 3.1933 - accuracy: 0.5474 - val_loss: 3.3306 - val_accuracy: 0.5307
Epoch 7/10
883/883 [==============================] - 64s 73ms/step - loss: 3.1111 - accuracy: 0.5595 - val_loss: 3.3090 - val_accuracy:

In [19]:
import numpy as np
sin_vocab = target_vectorization.get_vocabulary()
sin_index_lookup = dict(zip(range(len(sin_vocab)), sin_vocab))
max_decoded_sentence_length = 20

In [24]:
def decode_sequence(input_sentence):
  tokenized_input_sentence = source_vectorization([input_sentence])
  decoded_sentence = "[start]"
  for i in range(max_decoded_sentence_length):
    tokenized_target_sentence = target_vectorization(
      [decoded_sentence])[:, :-1]
    predictions = transformer(
      [tokenized_input_sentence, tokenized_target_sentence])
    sampled_token_index = np.argmax(predictions[0, i, :])
    sampled_token = sin_index_lookup[sampled_token_index]
    decoded_sentence += " " + sampled_token
    if sampled_token == "[end]":
      break
  return decoded_sentence

In [25]:

test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(20):
  input_sentence = random.choice(test_eng_texts)
  print("-")
  print(input_sentence)
  print(decode_sequence(input_sentence))


-
වාව් වාව් වාව් වාව් මම මත්ද්‍රව්‍ය අරගෙන නෑ අපිට සාක්‍ෂි තියෙනවා
[start] [UNK] [UNK] නෑ [UNK] මම [UNK] [UNK] නෑ ඒක [UNK] [UNK] [end]
-
ok the cold
[start] හරි උදේ [end]
-
trying on the flight  
[start] උත්සාහ කරන්න [end]
-
but there are chain murders in mumbai
[start] ඒත් මෙතන ඉන්නවා සාමාන්‍ය [UNK] [UNK] [end]
-
my goodness
[start] දෙවියනේ [end]
-
nathan
[start] දැනටමත් [end]
-
i don't have time for this right  
[start] මට මේක වෙලාවක් නෑ හරිද [end]
-
what are the gains and losses    
[start] මොකද [UNK] [UNK] [UNK] [end]
-
besides he's still a coward
[start] අනික එයා තාමත් [UNK] [end]
-
you didn't hear did you  
[start] ඔයා [UNK] නේද [end]
-
what woman could that be stoddard  
[start] ඒ [UNK] වෙන්න පුළුවන් ඒ [UNK] [end]
-
ok
[start] හරි [end]
-
why don't you listen to me  
[start] ඇයි ඔයා මට කියන දේ [end]
-
sacrificing lives   or equivalent knowledge  
[start] [UNK] [UNK] [UNK] [UNK] [UNK] [end]
-
are you comparing me to the dog  
[start] ඔයා මට මගේ බල්ලා එක්ක [end]
-
 where are we   